In [1]:
from multiprocessing import Process
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

import cv2


from subprocess import run, DEVNULL

from torch.nn import Sequential, Sigmoid
from torch import from_numpy, inference_mode
from torch.nn import DataParallel

from monai.networks.nets import SwinUNETR
from monai.inferers import sliding_window_inference

from pytorch_lightning import LightningModule

from zipfile import ZipFile
from zipfile import BadZipFile

import xmltodict

import os
from os.path import join, exists, getsize, isfile

In [4]:
class DownloadSARSentinel:
    def __init__(self, email, 
             password,
             product_list,
             satellite="sentinel_1"
                ):
        self.password = password
        self.email = email
        self.product_list = product_list
        self.satellite = satellite
        self.raw_folder = None
        self._query_result = None

    def query_result(self):
        if not isinstance(self._query_result, pd.DataFrame):
            self._query_result = self.query_sar()
        return self._query_result
        
    def query_sar(self):
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{name}')"
        query_result = pd.DataFrame()

        for product in self.product_list:
            json = requests.get(API_URL_NAME.format(name=product)).json()
            response_result = pd.DataFrame.from_dict(json['value'])

            if not response_result.empty:
                print("Found:", product)
                query_result = pd.concat([query_result, response_result])   
                #yield response_result
            else:
                print("Could not find: ", product)

        return query_result
        
    @staticmethod
    def create_folder(folder_path, original_name):
        if not folder_path:
            folder_path = join(os.getcwd(), original_name)
            if not exists(folder_path):
                os.mkdir(folder_path)

        assert exists(folder_path), f"Path not found: {folder_path}"
        return folder_path

    @staticmethod
    def is_downloaded(sar_name, folder):
        sar_path = join(folder, sar_name + ".zip")
        if exists(sar_path):
            #if getsize(sar_path) > 5E9: #ver o tamanho do arquivo para baixar arquivos incompletos
            return True
        return False

    def return_headers(self):
        token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'password',
            'username': self.email,
            'password': self.password,
            'client_id': 'cdse-public'
        }
        token_response = requests.post(token_url, headers=headers, data=data).json()
        token_url = token_response["access_token"]
        download_headers = {"Authorization": f"Bearer {token_url}"}
        return download_headers

    def download_products(self, folder, overwrite=False):
        download_url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

        for index, product in self.query_result().iterrows():
            path_product = join(folder, product["Name"] + ".zip")               
            if not DownloadSARSentinel.is_downloaded(product["Name"], folder) or overwrite:
                session = requests.Session()
                session.headers.update(self.return_headers())
                response = session.get(download_url.format(product_id=product["Id"]), headers=self.return_headers(), stream=True)
                total_size = int(response.headers.get('Content-Length', 0))
                progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f'Downloading: {product["Name"]}', leave=True)
    
                with open(path_product, "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                            progress_bar.update(len(chunk))
                progress_bar.reset()




        

In [5]:
QUERY_CSV_PATH = "/mnt/camobi_2/PHMG/new_img_acqusition/New_sar_img.csv"
FILE_NAME_COLUMN = "sar_file_name"
query_sar = pd.read_csv(QUERY_CSV_PATH, header=0)[FILE_NAME_COLUMN]
query_sar = query_sar[2:4]
#print(query_sar)

test = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=list(query_sar))

test.query_result()
test.download_products(folder="/mnt/camobi_2/PHMG/new_img_acqusition/raw_folder")

Found: S1A_IW_SLC__1SDV_20230716T214740_20230716T214806_049454_05F265_30F5.SAFE
Found: S1A_IW_SLC__1SDV_20230704T214739_20230704T214806_049279_05ECFB_A494.SAFE


In [6]:
class SentinelProduct:
    def __init__(self, name, file_path, rect_corners=None):
        self.name = name
        self.rect_corners = rect_corners
        self.file_path = file_path
        self.unzip_path = None

    def unzip(self, unzip_folder):
        assert exists(unzip_folder)
        
        self.unzip_path = join(unzip_folder, self.name) 

        if not self.name in os.listdir(self.unzip_path):
            try:
                print("Unziping:", self.name)
                with ZipFile(self.self.file_path, 'r') as zip_ref:
                    zip_ref.extractall(self.unzip_path)
            except BadZipFile:
                print(f"Imposible to unzip: {self.file_path}; Adding to download queue")
